In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Agam-Patel-DS/wine-quality-prediction-end-to-end.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Agam-Patel-DS"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3d17a240490c70ca0d48f408ad47e0542b365a0a"

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/workspaces/wine-quality-prediction-end-to-end'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from src.winequalityprediction.constants import *
from src.winequalityprediction.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Agam-Patel-DS/wine-quality-prediction-end-to-end.mlflow"
        )
        return model_evaluation_config

In [6]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config

    def eval_metric(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)
            (rmse,mae,r2)=self.eval_metric(test_y,predicted_qualities)

            scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            if tracking_url_type_store!="file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [9]:
config=ConfigurationManager()
model_evaluation_config=config.get_model_evaluation_config()
model_evaluation=ModelEvaluation(config=model_evaluation_config)
model_evaluation.log_into_mlflow()

[2025-02-03 08:01:41,624: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-03 08:01:41,625: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-03 08:01:41,628: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-03 08:01:41,628: INFO: common: Created directory at: artifacts]
[2025-02-03 08:01:41,629: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-02-03 08:01:42,321: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/02/03 08:01:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/02/03 08:01:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run flawless-toad-291 at: https://dagshub.com/Agam-Patel-DS/wine-quality-prediction-end-to-end.mlflow/#/experiments/0/runs/36949d52672e424ea5dc92195d95e8e9
🧪 View experiment at: https://dagshub.com/Agam-Patel-DS/wine-quality-prediction-end-to-end.mlflow/#/experiments/0
